## Examen Parcial de Mineria de Datos



```
# Estudiante : Widmar Raul Quispe Leon
# Codigo     : 171259
# Curso      : Mineria de Datos
```



In [1]:

# ----- Download and install pyspark and all requirements -----

# Download and install Java 8
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Apache Spark binary: This link can change based on the version. Update this link with the latest version before using
!wget -q https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz

# Unzip file
!tar -xvzf spark-3.2.0-bin-hadoop2.7.tgz

# Install findspark: Adds Pyspark to sys.path at runtime
!pip install -q findspark

# Install pyspark
!pip install pyspark

# Add environmental variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"

# findspark will locate spark in the system
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 http://ppa

Iniciamos Spark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("Hands-on PySpark on Google Colab") \
        .getOrCreate()

spark

# Algoritmo 1: Term Frecuency (TF)

Estructuramos la data y la mostramos

In [3]:
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext

# creamos el spark context
sc =SparkContext.getOrCreate()

# cargamos el archivo
file_name = "twitts.txt"
f = open(file_name)

# estructuramos la data
data = []
count = 1
for row in f:
    if row != "\n":
        row = row.replace("\n","")
        data.append((count,row))
        count += 1

rows = sc.parallelize(data)
rows.collect()


[(1, 'protester stage diein protest store nyc anger misplace retweet agree'),
 (2,
  'apple intraday comment update range premium user aapl stockaction trading stock mkt'),
 (3, 'ios app 2014 name elevate brain training iphone app'),
 (4, 'shit'),
 (5, 'founder attack boss ridiculous comment ad'),
 (6, 'evidence factcheckthis'),
 (7, 'educate'),
 (8, 'hard reach buy suicide squad comic_strip apple store turkey'),
 (9, 'delete music customer ipod'),
 (10,
  'apple intraday comment update range premium user aapl stockaction trading stock mkt'),
 (11, 'studio 45,000 outlet computer need battery future'),
 (12, 'apple great business aapl investwall aapl'),
 (13, 'dear love iphone plus lot great achievement iphone reboot day fail'),
 (14,
  'survey feedback iwatch positioning fashion accessory geek centric gadget'),
 (15, 'similar cloud backup picture onlinefootprint modicumofprudence'),
 (16, 'protester stage diein protest store nyc anger misplace retweet agree'),
 (17, 'fuck make shit pro

Mapeamos y mostramos sus TFs

In [4]:
map1 = rows.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])
reduce = map1.reduceByKey(lambda x,y:x+y)
tf = reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))
tf.collect()

[('protester', (1, 1)),
 ('stage', (1, 1)),
 ('diein', (1, 1)),
 ('protest', (1, 1)),
 ('store', (1, 1)),
 ('nyc', (1, 1)),
 ('anger', (1, 1)),
 ('misplace', (1, 1)),
 ('retweet', (1, 1)),
 ('agree', (1, 1)),
 ('apple', (2, 1)),
 ('intraday', (2, 1)),
 ('comment', (2, 1)),
 ('update', (2, 1)),
 ('range', (2, 1)),
 ('premium', (2, 1)),
 ('user', (2, 1)),
 ('aapl', (2, 1)),
 ('stockaction', (2, 1)),
 ('trading', (2, 1)),
 ('stock', (2, 1)),
 ('mkt', (2, 1)),
 ('ios', (3, 1)),
 ('app', (3, 2)),
 ('2014', (3, 1)),
 ('name', (3, 1)),
 ('elevate', (3, 1)),
 ('brain', (3, 1)),
 ('training', (3, 1)),
 ('iphone', (3, 1)),
 ('shit', (4, 1)),
 ('founder', (5, 1)),
 ('attack', (5, 1)),
 ('boss', (5, 1)),
 ('ridiculous', (5, 1)),
 ('comment', (5, 1)),
 ('ad', (5, 1)),
 ('evidence', (6, 1)),
 ('factcheckthis', (6, 1)),
 ('educate', (7, 1)),
 ('hard', (8, 1)),
 ('reach', (8, 1)),
 ('buy', (8, 1)),
 ('suicide', (8, 1)),
 ('squad', (8, 1)),
 ('comic_strip', (8, 1)),
 ('apple', (8, 1)),
 ('store', (8, 1

## Algoritmo 2: Inverse Document Frecuency (IDF)


### Mapeamos los valores de un TF

In [6]:
map3 = reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
map4 = map3.map(lambda x:(x[0],x[1][2]))
reduce2 = map4.reduceByKey(lambda x,y:x+y)


Reducimos por la llave

In [8]:
import math
from pyspark.sql.functions import *
idf = reduce2.map(lambda x: (x[0],math.log10(len(data)/x[1])))
# idf.collect()

### Mostramos el Term Frecuency - Inverse Document Frecuency (TF - IDF)

In [9]:
rdd = tf.join(idf)
rdd1 = rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()

rdd2 = rdd1.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
rdd2.toDF(["DocumentId","Token","TF","IDF","TF-IDF"]).show(100)

+----------+-----------------+---+------------------+------------------+
|DocumentId|            Token| TF|               IDF|            TF-IDF|
+----------+-----------------+---+------------------+------------------+
|         1|            stage|  1|1.3798696809532192|1.3798696809532192|
|         1|          protest|  1| 1.377146822617746| 1.377146822617746|
|         1|            store|  1| 1.105595616949241| 1.105595616949241|
|         1|              nyc|  1| 1.353380100659997| 1.353380100659997|
|         1|            anger|  1| 1.388142206919209| 1.388142206919209|
|         1|         misplace|  1| 1.390935107103379| 1.390935107103379|
|         1|          retweet|  1| 1.399423217328898| 1.399423217328898|
|         1|        protester|  1| 1.388142206919209| 1.388142206919209|
|         1|            diein|  1| 1.399423217328898| 1.399423217328898|
|         1|            agree|  1|1.3690792008697128|1.3690792008697128|
|         2|           update|  1|1.393746084437207

## Algoritmo 3: Bolsa de palabras o Bag of Words (BoW)

In [10]:
# Importamos la librerías a utilizar
from pyspark.sql.functions import explode, split, regexp_replace, col, lower
#Data
data = [
    ['a', 'Jan', 'John', 'This is a document'],
    ['b', 'Feb', 'Mary', 'A book by Mary'],
    ['c', 'Mar', 'Luke', 'Newspaper article'],
    ['d', 'Apr', 'Mark', None]
]
columns = ['Title', 'Month', 'Author', 'Document']
#Creando un data frame df
df = spark.createDataFrame(data, columns)
#Mostrando el df
df.show()
#BOG
df.select(explode(split(regexp_replace("Document", "[,.-]", " "), "\s+")).alias("word"))\
    .groupby(lower(col("word")).alias("lower"))\
    .count()\
    .show()

+-----+-----+------+------------------+
|Title|Month|Author|          Document|
+-----+-----+------+------------------+
|    a|  Jan|  John|This is a document|
|    b|  Feb|  Mary|    A book by Mary|
|    c|  Mar|  Luke| Newspaper article|
|    d|  Apr|  Mark|              null|
+-----+-----+------+------------------+

+---------+-----+
|    lower|count|
+---------+-----+
| document|    1|
|       by|    1|
|newspaper|    1|
|  article|    1|
|     mary|    1|
|       is|    1|
|        a|    2|
|     this|    1|
|     book|    1|
+---------+-----+



## Algoritmo 4: Tokenizacion

In [13]:
from pyspark.ml.feature import Tokenizer

import os.path

df = spark.createDataFrame([
    (0, '''Sombrero
I showed my masterpiece to the grown-ups, and asked them whether the drawing frightened them.
But they answered: "Frighten? Why should any one be frightened by a hat?"
My drawing was not a picture of a hat.
It was a picture of a boa constrictor digesting an elephant.
But since the grown-ups were not able to understand it, I made another drawing: I drew the inside of a boa constrictor, so that the grown-ups could see it clearly.
They always need to have things explained.
My Drawing Number Two looked like this:
Boa
The grown-ups' response, this time, was to advise me to lay aside my drawings of boa constrictors, whether from the inside or the outside, and devote myself instead to geography, history, arithmetic, and grammar.
That is why, at the age of six, I gave up what might have been a magnificent career as a painter.''')
    ], ['id', 'sentence'])

tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenized = tokenizer.transform(df)
tokenized.select('words').show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|words   

## Algoritmo 5: Normalizacion

In [12]:
# Importamos la librerías a utilizar
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors

#Crea dataframe
dataFrame = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.5, -1.0]),),
    (1, Vectors.dense([2.0, 1.0, 1.0]),),
    (2, Vectors.dense([4.0, 10.0, 2.0]),)
], ["id", "features"])

# Normaliza cada vector usando la Normalizer.
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)
l1NormData = normalizer.transform(dataFrame)
print("Normalizando....")
l1NormData.show()

Normalizando....
+---+--------------+------------------+
| id|      features|      normFeatures|
+---+--------------+------------------+
|  0|[1.0,0.5,-1.0]|    [0.4,0.2,-0.4]|
|  1| [2.0,1.0,1.0]|   [0.5,0.25,0.25]|
|  2|[4.0,10.0,2.0]|[0.25,0.625,0.125]|
+---+--------------+------------------+

